# Jobagent Investigation
1. Run all cells.
1. View report at the bottom.

In [ ]:
su = "tfs-cus-1"
start = "2019-07-20T16:00:00.0000000Z"
end = "2019-07-20T16:33:36.0000000Z"
url = "https://notebooksv2.azure.com/yaananth/projects/06OasuNRs6rK/delays.ipynb"
baseUrl = "https://notebooksv2.azure.com/yaananth/projects/06OasuNRs6rK"
service = "tfs"
hub = "Build"
locationName = "tfsprodcus1"

In [ ]:
# This isn't needed if you are bootstraping
#!pip install Kqlmagic --no-cache-dir --upgrade
#!pip install nimport --no-cache-dir --upgrade
#!pip install azure-kusto-notebooks --no-cache-dir --upgrade

In [ ]:
# Import the things we use

# Note you can also use kql https://docs.microsoft.com/en-us/azure/data-explorer/kqlmagic
# %kql is single line magic
# %%kql is cell magic

# https://nbviewer.jupyter.org/github/ipython/ipython/blob/4.0.x/examples/IPython%20Kernel/Rich%20Output.ipynb#HTML
# https://ipython.readthedocs.io/en/stable/inte/magics.html
from IPython.display import display, HTML, Markdown, Javascript, clear_output

# http://pandas-docs.github.io/pandas-docs-travis/user_guide/reshaping.html
import pandas as pd
from pandas import Series, DataFrame
from datetime import datetime, timedelta, timezone
from urllib.parse import urlencode, quote_plus
from requests.utils import requote_uri
import time
import numpy as np
from matplotlib import pyplot as plt
from nimport.utils import tokenize, open_nb
import json
import os
import calendar as cal
import concurrent.futures
from azure.kusto.notebooks import utils as akn

In [ ]:
params = {
    "su": su,
    "start": start,
    "end": end,
    "url": url,
    "baseUrl": baseUrl,
    "service": service
}
root = 'devops-pipelines' if os.path.basename(os.getcwd()) != 'devops-pipelines' else ''
    

In [ ]:
# todo: use from akn
def to_datetime(timestamp):
    s = timestamp[:23] + 'Z' # only allow 5 decimals of precision
    for f in ("%Y-%m-%d %H:%M:%S.%fZ", "%Y-%m-%dT%H:%M:%S.%fZ"):
        try:
            return datetime.strptime(s, f)
        except:
            pass
def get_time(timestamp, d):
    return int((cal.timegm(to_datetime(timestamp).timetuple()) + (d * 60)) * 1000)
# end todo

def getKustoQuery(csl_filename, params):

    return tokenize(os.path.join(queryPath, csl_filename), params)

queryPath = os.path.join(root, 'queries')

states = {
    "ja": "ja",
    "healthagent": "healthagent"
};

state = "";

In [ ]:
%%capture
%load_ext Kqlmagic

In [ ]:
# authenticate kusto client
# you will need to copy the token into a browser window for AAD auth. 
client = akn.get_client('https://vso.kusto.windows.net')

In [ ]:
q_slow = os.path.join(queryPath, "SlowActivities.csl")
q_sqlSlow = os.path.join(queryPath, "SlowSql.csl")

jaPath = os.path.join(queryPath, 'ja')
q_jobSql = os.path.join(jaPath, "JASqlTime.csl")
with concurrent.futures.ThreadPoolExecutor() as executor:
    p1 = executor.submit(akn.execute_file, client, 'VSO', q_slow, params)
    p2 = executor.submit(akn.execute_file, client, 'VSO', q_sqlSlow, params)
    p3 = executor.submit(akn.execute_file, client, 'VSO', q_jobSql, params)

q_slowResult_df = akn.to_dataframe_from_future(p1)

q_sqlSlowResult_df = akn.to_dataframe_from_future(p2)

q_jobSqlResult_df = akn.to_dataframe_from_future(p3)    


In [ ]:
q_slowResult_df

In [ ]:
q_sqlSlowResult_df

In [ ]:
q_jobSqlResult_df